# Get all Internet users for each country and language spokent from 1990 onwards

Input
- language codes: https://www.ethnologue.com/sites/default/files/LanguageCodes.tab
- number-of-internet-users-by-country:

Output
- LangID: ISO 639-3 standard for language identifiers
- CountryID:
- Language: Language spoken
- Year: Year data was gathered
- Internet_users: number of active internet users
- Name_country: Name of the main country where used.

In [8]:
import pandas as pd

In [9]:
users_country = pd.read_csv("../../data/number-of-internet-users-by-country.csv")
#users_country = users_country.drop(users_country.columns[1], axis=1)
users_country.columns = ["Country",'CountryID', "Year", "Internet_users"]

In [10]:
users_country.head()

,Country,CountryID,Year,Internet_users
0,Afghanistan,AFG,1990,0
1,Afghanistan,AFG,2001,990
2,Afghanistan,AFG,2002,1003
3,Afghanistan,AFG,2003,20272
4,Afghanistan,AFG,2004,25520


In [11]:
number_countries = len(users_country['Country'].unique())
number_countries

193

In [12]:
languages_country = pd.read_csv('../../data/languages_country.csv', sep=';')

In [13]:
languages_country = languages_country.drop(languages_country.columns[2:4], axis=1)
languages_country.columns = ['Country', 'Language', 'LangID']

In [14]:
languages_country.head()

,Country,Language,LangID
0,Afghanistan,Persian,4096
1,Afghanistan,Uzbek,4096
2,Ã…land Islands,Swedish,4096
3,Albania,Albanian,1052
4,Albania,English,4096


In [15]:
merged = users_country.merge(languages_country, how='left', on='Country')

In [16]:
merged.head()

,Country,CountryID,Year,Internet_users,Language,LangID
0,Afghanistan,AFG,1990,0,Persian,4096.0
1,Afghanistan,AFG,1990,0,Uzbek,4096.0
2,Afghanistan,AFG,2001,990,Persian,4096.0
3,Afghanistan,AFG,2001,990,Uzbek,4096.0
4,Afghanistan,AFG,2002,1003,Persian,4096.0


In [17]:
language_internet_users = merged[['LangID','Language', 'Year', 'Internet_users', 'Country', 'CountryID']].sort_values(by=['Language','Year']).reset_index(drop = True)

In [18]:
language_internet_users.head()

,LangID,Language,Year,Internet_users,Country,CountryID
0,4096.0,Afrikaans,1990,0,Namibia,NAM
1,1078.0,Afrikaans,1990,0,South Africa,ZAF
2,1078.0,Afrikaans,1991,5106,South Africa,ZAF
3,1078.0,Afrikaans,1992,15299,South Africa,ZAF
4,1078.0,Afrikaans,1993,45840,South Africa,ZAF


In [19]:
language_internet_users.to_csv('../../data/internet_users_per_language_1990_2017.csv') 

In [21]:
# output: index: lang code & year, cols=[number of internet users, country(?)]